In [71]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(0)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [72]:
test = pd.read_csv('/kaggle/input/titanic/test.csv')
train = pd.read_csv('/kaggle/input/titanic/train.csv')
train2 = train
test2 = test

In [73]:
from sklearn.feature_selection import mutual_info_classif

features_disc = train2.drop(['Age','Fare'], axis = 1)

def MI(x,y):
    MI_score = mutual_info_classif(x, y, random_state=0)
    MI_score = pd.Series(MI_score, index = x.columns)
    MI_score = MI_score.sort_values(ascending = True)
    
#    sns.set_style('whitegrid')
#    score = MI_score.sort_values(ascending = True)
    width = np.arange(len(MI_score))
    ticks = list(MI_score.index)
    plt.bar(width, MI_score)
    plt.yticks(ticks,(width))
    return MI_score

In [74]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

trainX = train2.drop(['Survived','Name','Ticket','Cabin'], axis=1)
trainY = train2.Survived

trainX['Age'] = train2.Age.fillna(train2.Age.mean())
test2['Age'] = test2.Age.fillna(test2.Age.mean())
trainX['Embarked'] = train2.Embarked.fillna(train2.Embarked.mode()[0])
test2['Fare'] = test2.Fare.fillna(test2.Fare.mean())

for coluna in ['Sex','Embarked']:
    for t in [trainX,test2]:
        if t[coluna].dtypes == 'object':
            label_encoder = LabelEncoder()
            valores_numericos = label_encoder.fit_transform(t[coluna])
    
            onehot_encoder = OneHotEncoder(sparse=False)
            inteiros = valores_numericos.reshape(len(valores_numericos),1)
            t[coluna] = onehot_encoder.fit_transform(inteiros)


In [75]:
MI(trainX, trainY)


In [77]:
fig, axs = plt.subplots(1,2, figsize = (18,7))

#Criando gráficos da distribuição de "Fare"
axs[0].set_title("Distribuição das Taxas")
sns.histplot(x = "Fare", kde = True, data = train2, ax = axs[0]);
axs[1].set_title("Sobreviventes por taxa")
sns.histplot(x = "Fare",hue = 'Survived', kde = True, data = train2, ax = axs[1]);


In [78]:
fig, axs = plt.subplots(2,2, figsize = (15,6))

axs[0,0].set_title('Sexo/Idade')
sns.histplot(x = 'Age',hue = 'Sex', kde = True, data = train2, ax = axs[0,0])

axs[0,1].set_title('Sexo/Idade dos Sobreviventes')
sns.histplot(x = 'Age',hue = 'Sex', kde = True, data = train2.loc[train2['Survived']==1], ax = axs[0,1])

axs[1,0].set_title('Distribuição de sobrevivência de homens')
sns.histplot(data = train2.loc[train2['Sex']=='male'], x = 'Age', hue = 'Survived', kde = True, ax = axs[1,0]);

axs[1,1].set_title('Distribuição de sobrevivência de mulheres')
sns.histplot(data = train2.loc[train2['Sex']=='female'], x = 'Age', hue = 'Survived', kde = True, ax = axs[1,1]);


In [79]:
train2 = trainX
test2['Name'] = test.Name
train2['Name'] = train.Name
train2['Survived'] = train.Survived

for t in [train2, test2]:
    titles= t["Name"].str.split(',',expand = True)[1].str.split('.',expand = True)[0]
    t['Title'] = titles
    t = t.drop('Name', axis = 1, inplace = True)
for t in [train2, test2]:
    t['Family'] = t['Parch']+t['SibSp']
    t['Alone'] = 0
    for i in range(len(t.Alone)):
        if t.loc[i,'Family'] == 0:
            t.loc[i,"Alone"] = 1

        

In [80]:
plt.scatter('Age','Fare', data=train2)

In [81]:
t = train2.loc[train2['Fare']>500]
t = train2.drop(t.index)
t = t[['Age','Fare','Survived']]
plt.scatter('Age','Fare', data=t)


In [82]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=6,n_init=20,random_state=0)
t["Fare_age"] = km.fit_predict(t)
t["Fare_age"] = t["Fare_age"].astype("int")
fig,axs = plt.subplots(1,2,figsize = (15,8))
sns.scatterplot(x='Age',y='Fare', data=t,hue=t["Fare_age"],ax=axs[0],palette = "rainbow_r")
sns.barplot(x='Fare_age',y='Survived', data=t,ax=axs[1])

In [83]:
train2['Fare_age'] = t.Fare_age
train2.loc[train2['Fare_age'].isnull()] = 5

In [84]:
from scipy import stats
from sklearn.preprocessing import MinMaxScaler

#Criando scaller 
scaler = MinMaxScaler(copy = False)
t_scaler = pd.DataFrame()
for feature in ['Fare','Age']:
        t_scaler[[feature]] = train2[[feature]]
        t_scaler[[feature]] = scaler.fit_transform(t_scaler[[feature]])

fig, axs = plt.subplots(1,2, figsize = (18,7))

sns.histplot(x = "Age", kde = True, data = t_scaler, ax = axs[0]);
sns.histplot(x = "Fare", kde = True, data = t_scaler, ax = axs[1]);

In [85]:
from sklearn.decomposition import PCA

#Realizando o PCA de "Fare" e "Age" 
pca = PCA()
pca2 = pca.fit_transform(t_scaler)
columns = ['PC1','PC2']
pca2 = pd.DataFrame(pca2, columns = columns)
loadings = pd.DataFrame(pca.components_.T, columns = columns, index = ["Fare","Age"])
print(loadings)
print("\n")
print(pca2)

In [86]:
MI(pca2, train2["Survived"])

In [87]:
from category_encoders import MEstimateEncoder

encoder = MEstimateEncoder(cols = ["Title"], m = 5)

title = encoder.fit_transform(train2["Title"], train2["Survived"])
print(title)



In [94]:
fig, ax = plt.subplots(figsize = (20,9))
sns.distplot(train2["Survived"], kde=False, norm_hist=True)
sns.kdeplot(title["Title"], color='r', ax=ax)
ax.set_xlabel("Survived")
ax.legend(labels=["Title", "Survived"]);

In [98]:
train2.Title = title
X_enc = train2.loc[:,['Fare_age','Title']]
y_enc = train2['Survived']
MI(X_enc, y_enc)